In [1]:
import pandas as pd
import numpy
import torch 
import numpy as np
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.append("..")
sys.path.append("/hpc2hdd/home/tianlangxue/XAI4AD/concept_gridlock/analyse_results")
from vis_utils import * 

from utils import pad_collate
from dataloader_comma import CommaDataset
from dataloader_nuscenes import NUScenesDataset
from collections import Counter
from model import VTN
import matplotlib.pyplot as plt 
from PIL import Image
import glob
import os
from utils import * 
import re
import plotly.graph_objects as go

-----------using concept datapath at:--- /hpc2hdd/home/tianlangxue/XAI4AD/concept_gridlock/scenarios/scenarios_small_100.txt
torch.Size([100, 77])


/hpc2hdd/home/tianlangxue/anaconda3/envs/OPT/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


using img_noise
current dataset: /hpc2hdd/home/tianlangxue/XAI4AD/comma2k19data/comma_test_w_desired_filtered_GaussNoiseV2.h5py
data_path----- /hpc2hdd/home/tianlangxue/XAI4AD/comma2k19data/comma_test_w_desired_filtered_GaussNoiseV2.h5py -----------------
using concept features


In [2]:

import sys
sys.path.append("/hpc2hdd/home/tianlangxue/XAI4AD/concept_gridlock")
from model import *
from module import * 
# sys.path.append("/hpc2hdd/home/tianlangxue/XAI4AD/concept_gridlock/New")
# from model_copy1 import *
# from model_copy2 import *
# from module_copy1 import  *
#define  pertubation and hyperparams
img_noise ="GaussNoiseV2"
multitask ="multitask"
bs = 1
dataset = "comma"
backbone = "none"
checkpoint_path = '/hpc2hdd/home/tianlangxue/XAI4AD/comma2k19data/ckpts_final/ckpts_final_comma_multitask_none_True_1/lightning_logs/version_52_5loss/checkpoints/epoch=0-step=120.ckpt'
model = VTN(multitask=multitask, backbone=backbone, concept_features=True, device = f"cuda:0", train_concepts=False)
# teachmodel = TVTN(multitask=multitask, backbone=backbone, concept_features=True, device = f"cuda:0", train_concepts=False)
testmodule = LaneModule.load_from_checkpoint(
    checkpoint_path,
    model=model,  # 传递模型参数
    # teachmodel=teachmodel,
    multitask=multitask,
    dataset=dataset,
    bs=bs,  # 传递批处理大小
    ground_truth="desired",
    # intervention=args.intervention_prediction,
    img_noise=img_noise,
    # dataset_path=args.dataset_path,
    # dataset_fraction=args.dataset_fraction
)

using concept features


view the concepts data

In [3]:

from tqdm import tqdm
concs = []
gt = []
conc_list = []
testmodule.to(f'cuda:0')
print("img noise:",commda_ds.img_noise)

for j, batch in tqdm(enumerate(dataloader_comma)):
    _, image_array, vego, angle, distance = batch
    img = image_array
    img, angle, distance, vego = img.to(f'cuda:0'), angle.to(f'cuda:0'), distance.to(f'cuda:0'), vego.to(f'cuda:0')
    logits, attns, concepts = testmodule(img, angle, distance, vego)
        # 将概念张量添加到列表中
    conc_list.append(concepts.cpu())  # 移动回CPU并添加到列表

    if j==0:
        # print(logits)

        print(logits[0].shape)
        print(logits[1].shape)
        print("----")
        print(img.shape)
        print(len(concepts))
        print(concepts.shape)
        # 将张量数据转换为 DataFrame
        logits_0_data = logits[0].detach().squeeze().cpu().numpy()  # 转换为 numpy 数组并移除多余的维度
        logits_1_data = logits[1].detach().squeeze().cpu().numpy()  # 转换为 numpy 数组并移除多余的维度
        angle_data =angle.detach().squeeze().cpu().numpy()
        dist_data =distance.detach().squeeze().cpu().numpy()
        df = pd.DataFrame({
            'logits_0': logits_0_data.flatten(),
            'logits_1': logits_1_data.flatten(),
            'angle_gt':angle_data,
            'dist_gt':dist_data
        })

        csv_file_path = 'cleanDCGlogits_data.csv'
        df.to_csv(csv_file_path, index=False)

        print(f"Data saved to {csv_file_path}")
        break
        


# thisimg =(thisimg.astype(float)*255)
# image = Image.fromarray(thisimg.astype('uint8'), 'RGB')
# image.show()
# conc_probs = torch.stack(conc_list)
# print(conc_probs.shape)




img noise: GaussNoiseV2


0it [00:03, ?it/s]

torch.Size([1, 240, 1])
torch.Size([1, 240, 1])
----
torch.Size([1, 240, 3, 224, 224])
1
torch.Size([1, 240, 100])
Data saved to cleanDCGlogits_data.csv
(224, 224, 3)


In [ ]:
# 存储concept_probs到文件
torch.save(conc_probs, '/hpc2hdd/home/tianlangxue/XAI4AD/concept_gridlock/my_concept_probs/conc_probs_tensors/distance_concept_text10%altered.pt')

calculate a wasserstein_distance between concepts data under different perturbations

In [ ]:
from scipy.stats import wasserstein_distance
import numpy as np
import torch

def load_tensor_from_file(file_path):
    return torch.load(file_path)

def softmax(x):
    """Compute the softmax of each element in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def calculate_EMD(distribution1, distribution2):
    dist1_np = distribution1.cpu().numpy()
    dist2_np = distribution2.cpu().numpy()
    frame_num = dist1_np.shape[0]

    # 计算并返回 EMD
    mean_emd = 0
    for frame in range(frame_num):
        mean_emd += wasserstein_distance(softmax(dist1_np[frame]), softmax(dist2_np[frame]))
    mean_emd /= frame_num
    return mean_emd

file_path1 = '/hpc2hdd/home/tianlangxue/XAI4AD/concept_gridlock/my_concept_probs/conc_probs_tensors/distance_concept_text10%altered.pt'  # 替换为第一个文件的路径
file_path2 = '/hpc2hdd/home/tianlangxue/XAI4AD/concept_gridlock/my_concept_probs/conc_probs_tensors/multi_concept_text10%alterd.pt'  # 替换为第二个文件的路径

probs1 = load_tensor_from_file(file_path1)
probs2 = load_tensor_from_file(file_path2)

probs1 = probs1.squeeze() 
probs2 = probs2.squeeze()


probs1 = probs1.to(probs2.device)


emd_values = []
for i in range(probs1.size(0)):

    emd = calculate_EMD(probs1[i], probs2[i])
    emd_values.append(emd)

# 打印结果
print(f"Earth Mover's Distance values for each sample: {emd_values}")
average_emd = np.mean(emd_values)
print(f"Average EMD: {average_emd}")

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(probs1[0])
plt.colorbar()

In [ ]:
print(softmax(probs1.cpu().numpy()[0, 0]))

In [ ]:
print(probs1-probs2)

In [ ]:
file_name1 = file_path1.split('/')[-1]
file_name2 = file_path2.split('/')[-1]
output_file_path = '/hpc2hdd/home/tianlangxue/XAI4AD/concept_gridlock/my_concept_probs/conceptsmetrics/EWD.txt' 
with open(output_file_path, 'a') as f:
    f.write(f"conceptprobs 1: {file_name1}\n")
    f.write(f"conceptprobs 2: {file_name2}\n")
    f.write(f"EMD values for each sample: {emd_values}\n")
    f.write(f"Average EMD: {average_emd}\n")

print(f"EWD results have been saved to {output_file_path}")